### Importing dependencies

In [1]:
import moviepy
import moviepy.editor
import numpy as np
import cv2 
import os
import random
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import import_ipynb
from scipy import stats
from keras.models import load_model
import pickle
import librosa

Using TensorFlow backend.


In [3]:
Testfilepath = '../Data/testing/happy_ramdas.mp4'
AudioFilepath = '../Data/testing/audio'
Framespath = '../Data/testing/frames'

### Extracting .wav and frames from .mp4 file

In [4]:
def extractDetails(path):
    video = moviepy.editor.VideoFileClip(path)
    audio = video.audio
    audio.write_audiofile(AudioFilepath+'/audio.wav')
    vidObj = cv2.VideoCapture(path) 
    count = 0
    # checks whether frames were extracted 
    success = 1
    while success: 
        try:             
            # vidObj object calls read and function extract frames 
            success, image = vidObj.read() 
            # Saves the frames with frame-count 
            cv2.imwrite(Framespath+"//frame%d.jpg" % count, image) 
            count += 1
        except:
            Done = True
extractDetails(Testfilepath)

MoviePy - Writing audio in ../Data/testing/audio/audio.wav


MoviePy - Done.


In [5]:
expressions = {'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sadness': 5, 'surprise': 6}

In [6]:
files = os.listdir(Framespath)
rdmnum = random.sample(range(0, len(files)), 9)
img = []
for ind in rdmnum:
    img.append(files[ind])

In [7]:
#detects the face in image using HAAR cascade then crop it then resize it and finally save it.
face_cascade = cv2.CascadeClassifier('../Data/training/haarcascade_frontalface_default.xml') 
def face_det_crop_resize(img_path):
    img = cv2.imread(img_path)
    try:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    except Exception as e:
        print(img_path)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x,y,w,h) in faces:
        face_clip = img[y:y+h, x:x+w]  #cropping the face in image
        cv2.imwrite(img_path, cv2.resize(face_clip, (350, 350)))  #resizing image then saving it
for image in img:
    face_det_crop_resize(Framespath+'/'+image)

### Loding images and audio files

In [8]:
img_data_list=[]
for image in img:
    input_img=cv2.imread(Framespath + '/'+ image )
    input_img_resize=cv2.resize(input_img,(256,256))
    img_data_list.append(input_img_resize)
visual_data = np.array(img_data_list)
visual_data = visual_data.astype('float32')
visual_data = visual_data/255

In [9]:
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccsscaled = np.mean(mfccs.T,axis=0)
    return mfccsscaled
features = []
audio_file = os.listdir(AudioFilepath)[0]
file_name = str(AudioFilepath)+'/'+str(audio_file)
features.append(extract_features(file_name))
audio_data = np.array(features).reshape(-1,40,1)    

### Loading the saved models

In [10]:
visual_model = load_model('../weights/visual_network.h5')
audio_model = load_model('../weights/audio_network.h5')
with open('../weights/ensemble.pkl', 'rb') as file:
    ensemble_model = pickle.load(file)
print('Loaded saved models')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Loaded saved models


### Predicting on the test data case

In [11]:
visualpredclass = visual_model.predict_classes(visual_data)
visualpredprobs = visual_model.predict(visual_data)
MajClassIndx = np.where(visualpredclass == stats.mode(visualpredclass)[0][0])
visualpredprob = visualpredprobs[random.choice(MajClassIndx[0])]

In [18]:
audiopredprob = audio_model.predict(audio_data, verbose=0)

In [20]:
vdf = pd.DataFrame([visualpredprob],columns=['angerVN','disgustVN','fearVN', 'joyVN', 'neutralVN', 'sadnessVN', 'surpriseVN'])
adf = pd.DataFrame(audiopredprob,columns=['angerAN','disgustAN','fearAN', 'joyAN', 'neutralAN', 'sadnessAN', 'surpriseAN'])

In [21]:
frames = [vdf,adf]
finaldf = pd.concat(frames,axis=1)
finaldf = finaldf.sample(frac=1).reset_index(drop=True)

In [22]:
predictedexp = ensemble_model.predict(finaldf)

In [23]:
for exp, label in expressions.items():
    if label == predictedexp[0]:
        print('Emotion Detected by Ensemble Model:', exp)

Emotion Detected by Ensemble Model: disgust
